In [1]:
import pandas as pd
import numpy as np
import os
import io
import sqlite3
from bs4 import BeautifulSoup

In [2]:
def extract_portfolio(bs_content, contextRef, fund_id):
    
    #Scrape the portfolio information
    port_soup = bs_content.find_all(['CodigoISIN',
                                     'InversionesFinancierasDescripcion',
                                     'InversionesFinancierasPorcentaje'], {'contextRef': [contextRef + '_ia', contextRef + '_ipp']})

    list_ISIN = []
    list_asset = []
    list_description = []
    list_weights_curr = []
    list_weights_past = []

    for i in range(len(port_soup)):
        if port_soup[i].name == 'CodigoISIN' and port_soup[i+1].name == 'InversionesFinancierasDescripcion' and port_soup[i+2].name == 'InversionesFinancierasPorcentaje' and port_soup[i+3].name == 'InversionesFinancierasPorcentaje':

            list_ISIN += [port_soup[i].text]
            i_aux_des = port_soup[i+1].text
            list_asset += [i_aux_des.split('|')[0]]
            list_description += [i_aux_des]
            list_weights_curr += [float(port_soup[i+2].text)/100]
            list_weights_past += [float(port_soup[i+3].text)/100]

    portfolio = pd.DataFrame(data={'ISIN': list_ISIN, 'Activo': list_asset, 'Descripción': list_description,'Peso Actual': list_weights_curr, 'Peso Anterior': list_weights_past})

    other_weights = 1 - portfolio[['Peso Actual', 'Peso Anterior']].sum()
    portfolio = portfolio.append(pd.DataFrame(data={'ISIN': np.nan, 'Activo': 'Tesorería y Otros', 'Descripción': 'Tesorería y Otros', 'Peso Actual': other_weights[0], 'Peso Anterior': other_weights[1]}, index=[0]), ignore_index=True)    
    portfolio['Fund ID'] = fund_id
    
    return portfolio

In [3]:
list_files = os.listdir('Info Pública')
list_files.remove('Funds DB')

all_prts = pd.DataFrame(columns=['ISIN', 'Activo', 'Descripción', 'Peso Actual', 'Peso Anterior', 'Fund ID'])

for i_file in list_files:
    
    try:
        #Convert xml format to Beautifulsoup
        content = []
        # Read the XML file        
        with open('Info Pública/' + i_file, 'rb') as file:
            i_bs_content = BeautifulSoup(file.read(), 'xml')        

        #contextRef is an attribute of each file
        #adding '_ia' makes reference to current value and '_ipp' makes reference to past value   
        i_contextRef = i_bs_content.find('context')['id'].replace('_da','')    

        i_port = extract_portfolio(i_bs_content, i_contextRef, i_file.replace('.xml',''))

        all_prts = pd.concat([all_prts, i_port], ignore_index=True)
    
    except:
        continue
    
all_prts.to_pickle('portfolios.pickle')    
all_prts.head(10)

,ISIN,Activo,Descripción,Peso Actual,Peso Anterior,Fund ID
0,ES00000128H5,RFIJA,RFIJA|SPAIN GOVERNMENT|1.30|2026-10-31,0.0006,0.0006,103-1081
1,ES0000012G34,RFIJA,RFIJA|SPAIN GOVERNMENT|1.25|2030-10-31,0.0006,0.0006,103-1081
2,ES0200002055,RFIJA,RFIJA|ADIF ALTA VELOCI|0.55|2030-04-30,0.0020,0.0019,103-1081
3,XS1645495349,RFIJA,RFIJA|CAIXABANK|2.75|2028-07-14,0.0020,0.0020,103-1081
4,ES0265936007,RFIJA,RFIJA|ABANCA CORP BANC|6.12|2029-01-18,0.0020,0.0021,103-1081
5,ES0343307015,RFIJA,RFIJA|KUTXABANK SA|0.50|2024-09-25,0.0019,0.0019,103-1081
6,ES0280907017,RFIJA,RFIJA|UNICAJA BANCO SA|2.88|2029-11-13,0.0017,0.0019,103-1081
7,XS2103013210,RFIJA,RFIJA|RED ELECTRICA FI|0.38|2028-07-24,0.0020,0.0020,103-1081
8,ES0305063010,RFIJA,RFIJA|SIDECU SA|5.00|2025-03-18,0.0021,0.0020,103-1081
9,ES0244251015,RFIJA,RFIJA|IBERCAJA |2.75|2030-07-23,0.0017,0.0018,103-1081


In [7]:
con = sqlite3.connect('Portfolios.db')
cur = con.cursor()
#cur.execute('DROP TABLE IF EXISTS Portfolios')
cur.execute('CREATE TABLE Portfolios (ISIN TEXT, Activo TEXT, Descripción TEXT, Peso_Actual REAL, Peso_Anterior REAL, Fund_ID TEXT)')

records = list(all_prts.to_records(index=False))

cur.executemany('INSERT INTO Portfolios VALUES (?, ?, ?, ?, ?, ?)', records)

con.commit()

con.close()

In [4]:
assets_caract_df = pd.DataFrame(data={'ISIN': all_prts['ISIN'].dropna().unique()})
list_desc = []
list_asset = []
for i_ISIN in list(all_prts['ISIN'].dropna().unique()):
    list_desc += [all_prts.loc[all_prts['ISIN'].isin([i_ISIN])]['Descripción'].values[0]]
    list_asset += [all_prts.loc[all_prts['ISIN'].isin([i_ISIN])]['Activo'].values[0]]
    
assets_caract_df['Descripción'] = list_desc
assets_caract_df['Activo'] = list_asset

assets_caract_df.to_pickle('AssetsFeatures.pickle') 

con = sqlite3.connect('AssetsFeatures.db')
cur = con.cursor()
#cur.execute('DROP TABLE IF EXISTS AssetsFeatures')
cur.execute('CREATE TABLE AssetsFeatures (ISIN TEXT, Descripción TEXT, Activo TEXT)')

records = list(assets_caract_df.to_records(index=False))

cur.executemany('INSERT INTO AssetsFeatures VALUES (?, ?, ?)', records)

con.commit()

con.close()

assets_caract_df.head(10)

,ISIN,Descripción,Activo
0,ES00000128H5,RFIJA|SPAIN GOVERNMENT|1.30|2026-10-31,RFIJA
1,ES0000012G34,RFIJA|SPAIN GOVERNMENT|1.25|2030-10-31,RFIJA
2,ES0200002055,RFIJA|ADIF ALTA VELOCI|0.55|2030-04-30,RFIJA
3,XS1645495349,RFIJA|CAIXABANK|2.75|2028-07-14,RFIJA
4,ES0265936007,RFIJA|ABANCA CORP BANC|6.12|2029-01-18,RFIJA
5,ES0343307015,RFIJA|KUTXABANK SA|0.50|2024-09-25,RFIJA
6,ES0280907017,RFIJA|UNICAJA BANCO SA|2.88|2029-11-13,RFIJA
7,XS2103013210,RFIJA|RED ELECTRICA FI|0.38|2028-07-24,RFIJA
8,ES0305063010,RFIJA|SIDECU SA|5.00|2025-03-18,RFIJA
9,ES0244251015,RFIJA|IBERCAJA |2.75|2030-07-23,RFIJA
